In [ ]:
!pip install plotly==5.3.1

In [ ]:
!pip install SimpleITK

In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import SimpleITK as sitk
import plotly.express as px
import plotly.graph_objects as go
import math

In [4]:
def load_img_from_tiff(path2img):
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

In [ ]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Galleria Piastra Modello 1 100x100 Echi1-10x.tif"
data = load_img_from_tiff(path)
plt.imshow(data[0])

In [ ]:
data[0]

In [ ]:
for i in range(len(data)):
    data[i] = data[i]*(255.0/data[i].max())
px.imshow(data[0])

In [ ]:
fig = px.imshow(data, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

In [9]:
blurred_data = np.zeros_like(data)
for i in range(len(blurred_data)):
    blurred_data[i] = cv2.GaussianBlur(data[i], (3,3),3)

# plt.imshow(blurred_data[0])
# plt.show()

thresholded_data = np.zeros_like(data)
for i in range(len(thresholded_data)):
    bin, thresholded_data[i] = cv2.threshold(blurred_data[i],185, 255, cv2.THRESH_BINARY)

# plt.imshow(thresholded_data[0])
# plt.show()

In [ ]:
im_2 = np.array([data, blurred_data, thresholded_data])
fig = px.imshow(im_2, 
                animation_frame=1, 
                facet_col=0, 
                color_continuous_scale ="gray")
fig.show()

In [ ]:
from skimage.morphology import skeletonize, thin, medial_axis
skeleton = skeletonize(thresholded_data, method="lee")

# medial_axis_skel = medial_axis(thresholded_data)

medial_axis_skel = np.zeros_like(data)
for i in range(len(thresholded_data)):
    
    skel, distance = medial_axis(thresholded_data[i], return_distance=True)
    dist_on_skel = distance * skel
    medial_axis_skel[i]=skel

In [ ]:
im_3 = np.array([thresholded_data,medial_axis_skel*255])
fig = px.imshow(im_3, 
                animation_frame=1, 
                facet_col=0, 
                color_continuous_scale ="gray")
fig.show()

In [36]:
!mkdir animation3

In [ ]:
#sbs matplotlib, large output warning!
plt.rcParams['figure.figsize'] =14, 5

for i in range(len(data)):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
    ax1.imshow(data[i], cmap='gray')
    ax1.set_xlabel("Raw image  z-axis slice: "+str(i+1))
    ax2.imshow(blurred_data[i], cmap='gray')
    ax2.set_xlabel("Gaussian blurred")
    ax3.imshow(thresholded_data[i], cmap='gray')
    ax3.set_xlabel("Thresholded")
    ax4.imshow(medial_axis_skel[i]*255, cmap='gray')
    ax4.set_xlabel("Medial axis")
    savename="/content/animation3/"+str(i)+".png"
    fig.savefig(savename)


In [ ]:
!zip -r animation3.zip animation3/